<a href="https://colab.research.google.com/github/gomdoori/AI/blob/main/DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim

In [ ]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader
import torchvision.datasets as datasets

In [ ]:
# 디바이스 할당
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
image_sz = 64
fake_sz = 100
lat_dimension = 64

In [ ]:
transform = transforms.Compose([transforms.Resize((image_sz,image_sz)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,),(0.5,)),
])

In [ ]:
train_dataset = datasets.MNIST(
    root = './data',
    train = True,
    transform = transform,
    download = True
)

In [ ]:
batch_size = 128

train_loader = DataLoader(
    train_dataset,
    batch_size = batch_size,
    shuffle = True
)

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        # 생성자를 구성하는 층 정의
        self.gen = nn.Sequential(
            nn.ConvTranspose2d(fake_sz, 512, kernel_size=4),
            nn.BatchNorm2d(512),
            nn.ReLU(),

            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),

            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.ConvTranspose2d(64, 1, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self,x):
        return self.gen(x)

In [ ]:
class Diciriminator(nn.Module):
    def __init__(self):
        super(Diciriminator, self).__init__()
        # 판별자를 구성하는 층의 정의
        self.disc = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),

            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),

            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),

            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),

            nn.Conv2d(512, 1, kernel_size=4),
            nn.Sigmoid()
        )
    def forward(self,x):
        return self.disc(x)

In [ ]:
generator = Generator().to(device)
discriminator = Diciriminator().to(device)
print(generator)
print(discriminator)

In [ ]:
optim_g = optim.Adam(generator.parameters(),lr=0.0001,betas=(0.5,0.999))
optim_d = optim.Adam(discriminator.parameters(),lr=0.0001,betas=(0.5,0.999))

In [ ]:
criterion = nn.BCELoss() # Binary Classification에 사용되는 손실함수

losses_g = [] # 에포크마다 생성자 오차 저장
losses_d = [] # 에포크마다 판별자 오차 저장

학습 반복문 부분

In [ ]:
from torchvision.utils import save_image # 이미지 저장 라이브러리

In [ ]:
epochs = 25
for epoch in range(epochs):
  loss_g = 0.0
  loss_d = 0.0
  for idx, data in enumerate(train_loader):
    # 판별자 학습 함수 부분
    optim_d.zero_grad()
    image, _ = data
    data_real = image.to(device)
    b_size = data_real.shape[0]
    real_label = torch.ones(b_size).to(device)
    fake_label = torch.zeros(b_size).to(device)

    output_real = discriminator(data_real)
    loss_real = criterion(torch.squeeze(output_real), real_label)

    noise = torch.randn(b_size, fake_sz, 1, 1).to(device)
    data_fake = generator(noise)
    output_fake = discriminator(data_fake.detach())
    loss_fake = criterion(torch.squeeze(output_fake), fake_label)

    loss_total = (loss_real + loss_fake) / 2
    loss_total.backward()
    optim_d.step()

    # 생성자 학습 함수 부분
    optim_g.zero_grad()
    output = discriminator(data_fake)
    g_loss = criterion(torch.squeeze(output), real_label)
    g_loss.backward()
    optim_g.step()

    loss_g += g_loss
    loss_d += loss_total

    # 생성자, 판별자 손실 함수 저장
    epoch_loss_g = loss_g / idx
    epoch_loss_d = loss_d / idx
    losses_g.append(epoch_loss_g)
    losses_d.append(epoch_loss_d)
    print(f"Epoch {epoch} of {epochs}")
    print(f"Generator Loss: {epoch_loss_g:.8f}, Discriminator Loss: {epoch_loss_d:.8f}")

    # 각 epoch마다 이미지 저장
    save_image(data_fake[25],
               f'{epoch}.png',
               nrow=5,
               normalize=True)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Epoch 9 of 25
Generator Loss: 3.03623581, Discriminator Loss: 0.29363659
Epoch 9 of 25
Generator Loss: 3.03743625, Discriminator Loss: 0.29277837
Epoch 9 of 25
Generator Loss: 3.03876328, Discriminator Loss: 0.29191816
Epoch 9 of 25
Generator Loss: 3.04319167, Discriminator Loss: 0.29109150
Epoch 9 of 25
Generator Loss: 3.04140925, Discriminator Loss: 0.29033318
Epoch 9 of 25
Generator Loss: 3.04556417, Discriminator Loss: 0.28956053
Epoch 9 of 25
Generator Loss: 3.04869890, Discriminator Loss: 0.28873923
Epoch 9 of 25
Generator Loss: 3.04773998, Discriminator Loss: 0.28794172
Epoch 9 of 25
Generator Loss: 3.05088639, Discriminator Loss: 0.28713647
Epoch 9 of 25
Generator Loss: 3.05454850, Discriminator Loss: 0.28630668
Epoch 9 of 25
Generator Loss: 3.05606627, Discriminator Loss: 0.28549060
Epoch 9 of 25
Generator Loss: 3.05910540, Discriminator Loss: 0.28461328
Epoch 9 of 25
Generator Loss: 3.06308651, Discriminator Loss: 0.28374887
Epoch 9 of 25
G

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------   
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------   
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
trans = transforms.Compose([transforms.Resize((image_sz,image_sz)),
                                transforms.ToTensor(),
                                transforms.CenterCrop(image_sz),
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        # 생성자를 구성하는 층의 정의
        self.gen = nn.Sequential(
            nn.ConvTranspose2d(fake_sz, 512, kernel_size=4, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(0.2),

            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1 , bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(0.2),

            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1 , bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(0.2),

            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(0.2),

            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1, bias=False),
            nn.Sigmoid()
        )
    def forward(self,x):
        return self.gen(x)

In [ ]:
class Diciriminator(nn.Module):
    def __init__(self):
        super(Diciriminator,self).__init__()
        # 판별자를 구성하는 층의 정의
        self.disc = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),

            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),

            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),

            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2),

            nn.Conv2d(512, 1, kernel_size=4),
            nn.Sigmoid()
        )
    def forward(self,x):
        return self.disc(x)

In [ ]:
# 가중치 초기화 설정
def weights_init(m):
  classname = m.__class__.__name__
  if classname.find('Conv') != -1:
    nn.init.normal_(m.weight.data, 0.0, 0.02)
  elif classname.find('BatchNorm') != -1:
    nn.init.normal_(m.weight.data, 1.0, 0.02)
    nn.init.constant_(m.bias.data, 0)

genrator = Generator().to(device)
genrator.apply(weights_init)
discriminator = Diciriminator().to(device)
discriminator.apply(weights_init)
print(genrator)
print(discriminator)